In [1]:
from snorkel.labeling import labeling_function
import json
import os
import numpy as np
import imageio.v2 as imageio
from LF_library import *
from LF_deep_utils import *
from LF_utils import *

In [ ]:
# import json

# # File paths
# file1_path = './vlm_responses/finla_results_llama3.2-vision_90b.json'
# file2_path = './vlm_responses/test_data/test_results_llama3.2-vision_90b.json'
# output_path = './vlm_responses/llama_merged.json'

# # Load the JSON files
# with open(file1_path, 'r') as file1:
#     data1 = json.load(file1)

# with open(file2_path, 'r') as file2:
#     data2 = json.load(file2)

# # Modify keys in the second JSON
# modified_data2 = {f"Dev-{key}": value for key, value in data2.items()}

# # Merge the dictionaries
# merged_data = {**data1, **modified_data2}

# # Save the merged dictionary into a new JSON file
# with open(output_path, 'w') as output_file:
#     json.dump(merged_data, output_file, indent=4)

# print(f"Merged JSON saved to {output_path}")


# Defining the Labelers

In [2]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

@labeling_function()
def Qwen(image_name):
    file_path = './vlm_responses/qwen_merged.json'
    # llava_7b_results = 
    # path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(file_path, 'r') as file:
        data = json.load(file)

    # for key in data:
    #     if data[key] == '4':
    #         data[key] = 1
    #     else:
    #         data[key] = 0

    return data[image_name] if data[image_name] is not None else -1

@labeling_function()
def Llama3_vision(image_name):
    root_path = './vlm_responses/llama_merged.json'
    # llava_7b_results = 'finla_results_llama3.2-vision_90b.json'
    # path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(root_path, 'r') as file:
        data = json.load(file)

    for key in data:
        if data[key] == ["4"]:
            data[key] = 1
        else:
            data[key] = 0

    return data[image_name] if data[image_name] is not None else -1


@labeling_function()
def deeplearning(image_name):
    root_path = './vlm_responses/deeplearning_merged.json'
    # llava_7b_results = './train_deep_learning_labels.json'
    # path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(root_path, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1

@labeling_function()
def line_detection(image_name):
    root_path = './vlm_responses/line_detection_merged.json'
    # llava_7b_results = './train_line_detection_labels.json'
    # path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(root_path, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1


@labeling_function()
def superpixel_detection(image_name):
    root_path = './vlm_responses/superpixel_merged.json'
    # llava_7b_results = './train_super_pixel_labels.json'
    # path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(root_path, 'r') as file:
        data = json.load(file)

    for key in data:
        if data[key] == -1:
            data[key] = 1

    return data[image_name] if data[image_name] is not None else -1
    

In [3]:
superpixel_detection("1005")

1

In [4]:
train_data_json_path = './vlm_responses/finla_results_llama3.2-vision_90b.json'
# dev_data_json_path = '../prompting_framework/prompting_results/hateful/simplified_dev.json'

with open(train_data_json_path, 'r') as file:
    train_data = json.load(file)

# Extract and pad image names, ensuring they are 5 digits long before the '.png'
train_image_names = []
for key in train_data:
    print(key)
    print(train_data[key])
    break

0
['4']


# Train Dataset

In [5]:
train_data_json_path = './vlm_responses/finla_results_llama3.2-vision_90b.json'
dev_data_json_path = './vlm_responses/test_data/Testing_Set.json'

with open(train_data_json_path, 'r') as file:
    train_data = json.load(file)

# Extract and pad image names, ensuring they are 5 digits long before the '.png'
train_image_names = []
for entry in train_data:
    # img_name, ext = entry['img'].split('.')
    # padded_img_name = img_name.zfill(5)  # Pad the image name to 5 digits
    train_image_names.append(str(entry))

with open(dev_data_json_path, 'r') as file:
    dev_data = json.load(file)
    
dev_image_names = []
Y_dev = []
for entry in dev_data:
    Y_dev.append(int(int(dev_data[entry]['label'])/4))
    # img_name, ext = entry['img'].split('.')
    # padded_img_name = img_name.zfill(5)  # Pad the image name to 5 digits
    dev_image_names.append(f"Dev-{entry}")

print(f"There are {len(train_image_names)} images in the Train set.")
print(f"There are {len(dev_image_names)} images in the dev set.")
print(f"There are {len(Y_dev)} labels in the dev set.")


There are 8518 images in the Train set.
There are 311 images in the dev set.
There are 311 labels in the dev set.


In [ ]:
Qwen(dev_image_names[0])

In [ ]:
dev_image_names

In [ ]:
train_image_names

# Applying the LFs

In [31]:
from snorkel.labeling import LFApplier

list_of_all_the_models = ['Qwen', 'Llama3_vision', 'deeplearning', 'line_detection', 'superpixel_detection'
       ]

lfs = [Qwen, Llama3_vision, deeplearning, line_detection, superpixel_detection
       ]
# lfs = [Qwen, Llama3_vision, deeplearning
#        ]

applier = LFApplier(lfs)

In [32]:
from snorkel.labeling import LFAnalysis

L_dev = applier.apply(dev_image_names)
L_train = applier.apply(train_image_names)

311it [00:02, 119.87it/s]
8518it [01:12, 117.25it/s]


In [33]:
Y_dev = np.array(Y_dev)
LFAnalysis(L_dev, lfs).lf_summary(Y=Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
Qwen,0,"[0, 1]",1.0,1.0,0.874598,222,89,0.713826
Llama3_vision,1,"[0, 1]",1.0,1.0,0.874598,200,111,0.643087
deeplearning,2,"[0, 1]",1.0,1.0,0.874598,263,48,0.845659
line_detection,3,"[0, 1]",1.0,1.0,0.874598,139,172,0.446945
superpixel_detection,4,"[0, 1]",1.0,1.0,0.874598,138,173,0.443730


F1 of the labelers:

In [34]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score
import numpy as np

def calculate_metrics(y_true, y_pred, abstain_class=-1):
    # Filter out samples where prediction is -1
    valid_indices = y_pred != abstain_class
    y_true_filtered = y_true[valid_indices]
    y_pred_filtered = y_pred[valid_indices]

    # Compute metrics
    conf_matrix = confusion_matrix(y_true_filtered, y_pred_filtered)
    precision = precision_score(y_true_filtered, y_pred_filtered)
    recall = recall_score(y_true_filtered, y_pred_filtered)
    f1 = f1_score(y_true_filtered, y_pred_filtered)
    accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
    cohen_kappa = cohen_kappa_score(y_true_filtered, y_pred_filtered)

    return {
        'Confusion Matrix': conf_matrix,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'cohen_kappa' : cohen_kappa
    }

Visualization:

In [35]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, f1_score

# Example ground truth and predictions for six models
# Replace these arrays with actual predictions from each model
y_true = Y_dev
predictions = {}

for i in range(L_dev.shape[1]):
    predictions[list_of_all_the_models[i]] = L_dev[:,i]
    
# Create a DataFrame to store confusion matrix results and metrics
confusion_data = []

for model_name, y_pred in predictions.items():
    
    y_pred = np.array(y_pred)
    # Confusion Matrix
    metrics = calculate_metrics(Y_dev, y_pred)
    
    tn, fp, fn, tp = metrics['Confusion Matrix'].ravel()
    precision = metrics['Precision']
    recall = metrics['Recall']
    f1 = metrics['F1 Score']
    accuracy = metrics['Accuracy']
    cohen_kappa = metrics['cohen_kappa']
    # Append data
    confusion_data.append([
        model_name, tn, fp, fn, tp, 
        recall, precision, accuracy, f1, cohen_kappa
    ])

# Convert to a DataFrame for display
confusion_df = pd.DataFrame(confusion_data, columns=[
    'Model', 'True Negative', 'False Positive', 'False Negative', 'True Positive', 
    'Recall', 'Precision', 'Accuracy', 'F1 Score', 'cohen_kappa'
])

# Display the table with confusion matrix and metrics
confusion_df


,Model,True Negative,False Positive,False Negative,True Positive,Recall,Precision,Accuracy,F1 Score,cohen_kappa
0,Qwen,55,79,10,167,0.943503,0.678862,0.713826,0.789598,0.377566
1,Llama3_vision,90,44,67,110,0.621469,0.714286,0.643087,0.664653,0.287124
2,deeplearning,111,23,25,152,0.858757,0.868571,0.845659,0.863636,0.685872
3,line_detection,34,100,72,105,0.593220,0.512195,0.446945,0.549738,-0.157034
4,superpixel_detection,99,35,138,39,0.220339,0.527027,0.443730,0.310757,-0.037366


# Optimization for Label Model

In [36]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=False)
label_model.fit(L_train, n_epochs=5000, log_freq=500, seed=12)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:01<00:00, 3178.09epoch/s]


In [ ]:
label_model.get_weights()

In [37]:
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds

probs_dev = label_model.predict_proba(L_dev)
preds_dev = probs_to_preds(probs_dev)

metrics = calculate_metrics(Y_dev, preds_dev)
for metric, value in metrics.items():
    print(f"{metric}: {value}")

Confusion Matrix: [[ 92  42]
 [ 44 133]]
Precision: 0.76
Recall: 0.751412429378531
F1 Score: 0.7556818181818182
Accuracy: 0.7234726688102894
cohen_kappa: 0.4371869870796684


In [ ]:
L_train.shape[1]

Experimenting with different number of LFs

In [14]:
all_f1s = []
number_of_epochs = 100
for i in range(3, L_train.shape[1]+1):

    
    label_model = LabelModel(cardinality=2, verbose=False)
    label_model.fit(L_train[:,:i], Y_dev, n_epochs=number_of_epochs, log_freq=500, seed=12)
    
    probs_dev = label_model.predict_proba(L_dev[:,:i])
    preds_dev = probs_to_preds(probs_dev)

    metrics = calculate_metrics(Y_dev, preds_dev)
    all_f1s.append(metrics['F1 Score'])
    for metric, value in metrics.items():
        print(f"{metric}: {value}")
    print(f'trained with the LFs = {list_of_all_the_models[:i]}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3724.30epoch/s]

Confusion Matrix: [[ 91  43]
 [ 19 158]]
Precision: 0.7860696517412935
Recall: 0.8926553672316384
F1 Score: 0.8359788359788359
Accuracy: 0.8006430868167203
cohen_kappa: 0.5844754762520472
trained with the LFs = ['Qwen', 'Llama3_vision', 'deeplearning']


In [ ]:
import matplotlib.pyplot as plt
x = np.array([4, 5, 6])

plt.figure(figsize=(8, 5))
plt.plot(x, all_f1s, marker='s', linestyle='-', color='b')
plt.title(f'The Label Model Performance After Optimizing For {number_of_epochs} Iterations')
plt.xlabel('Number of LFs')
plt.ylabel('F1 Score')
plt.grid(True)
# plt.savefig('HM_LM_1000_4-7.png')
plt.show()

In [ ]:
all_f1s = []
# number_of_epochs = [10, 20, 50, 80, 100, 500, 1000, 5000]
number_of_epochs = range(1,100)
for epochs in number_of_epochs:

    
    label_model = LabelModel(cardinality=2, verbose=False)
    label_model.fit(L_train, Y_dev, n_epochs=epochs, log_freq=500, seed=12)
    
    probs_dev = label_model.predict_proba(L_dev)
    preds_dev = probs_to_preds(probs_dev)

    metrics = calculate_metrics(Y_dev, preds_dev)
    all_f1s.append(metrics['F1 Score'])
    # for metric, value in metrics.items():
    #     print(f"{metric}: {value}")
    # print(f'trained with the LFs = {list_of_all_the_models[:i]}')

In [ ]:
import matplotlib.pyplot as plt
x = np.array(number_of_epochs)

plt.figure(figsize=(8, 5))
plt.plot(x, all_f1s, marker='s', linestyle='-', color='b')
plt.title(f'The Label Model Performance After Optimizing For {number_of_epochs} Iterations')
plt.xlabel('Number of LFs')
plt.ylabel('F1 Score')
plt.grid(True)
# plt.savefig('HM_LM_1000_4-7.png')
plt.show()

In [ ]:
all_f1s[1:]

# Training the End Model

The dataloader for PyTorch

In [15]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Custom dataset class for loading images
class ImageDataset(Dataset):
    def __init__(self, image_names, root_dir, labels, target_dists, val=False, transform=None):
        """
        Args:
            image_names (list): List of image file names.
            root_dir (string): Directory where images are stored.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.data = []
        self.tile_numbers = []
        self.val = val
        
        self.image_names = image_names
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.target_dists = target_dists
        tile_files = [f for f in os.listdir(self.root_dir) if f.endswith('.tif')]
        # neg_files = [f for f in os.listdir(neg_dir) if f.endswith('.tif')]
        tile_dict = self.group_files_by_tile(tile_files)
        
        self.store_tiles(tile_dict, self.root_dir)

    
    def group_files_by_tile(self, files):
        tile_dict = {}
        # print(len(files))
        for file in files:
            tile_number = file.split('_')[-1].split('.')[0]
            if tile_number not in tile_dict:
                tile_dict[tile_number] = []
            tile_dict[tile_number].append(file)
        # Only include complete groups
        # return [tile for tile in tile_dict.values() if len(tile) == 6]
        # return [tile for tile in tile_dict.values()]
        # print(tile_dict)
        return tile_dict

    def store_tiles(self, tile_dict, directory):
        for tile_number in self.image_names:
            # print("----")
            # print(tile_number)
           
            self.data.append([os.path.join(directory, f) for f in sorted(tile_dict[tile_number])])

            # else:
            #     self.data.append([os.path.join(directory, f) for f in sorted(tile_dict[])])
            self.tile_numbers.append(tile_number)
    
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        # Build the full path of the image file
        # img_name = os.path.join(self.root_dir, self.image_names[idx])
        # label = self.labels[idx]
        # target_dist = self.target_dists[idx]
        # image = Image.open(img_name).convert('RGB')  # Load image as RGB

        # # Apply any transformations (e.g., resize, normalization)
        # if self.transform:
        #     image = self.transform(image)
        
        image_paths = self.data[idx]
        images_1 = [imageio.imread(img_path).astype('uint8') for img_path in self.data[idx]]
        images = [transforms.functional.to_pil_image(image) for image in images_1]
        if self.transform:
            images = [self.transform(image) for image in images]

        label = self.labels[idx]
        target_dist = self.target_dists[idx]
        
        tile_number = self.tile_numbers[idx]
        # label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return images, torch.tensor(label), torch.tensor(target_dist), tile_number, image_paths

        # return image, torch.tensor(label), torch.tensor(target_dist)

In [38]:

root_dir = "/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/combined_folder_true_rgb/"
validation_root_dir = "/home/macula/SMATousi/cluster/docker-images/ollama/MO+IA_test_data_numbered/"

transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224 (example)
        transforms.ToTensor(),          # Convert images to PyTorch tensors
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean/std
    ])

probs_dev = label_model.predict_proba(L_train)
label_model_predictions = probs_to_preds(probs_dev)

In [39]:
dev_image_names_modified = [entry.split('-')[-1] for entry in dev_image_names]
print(dev_image_names_modified)

['415', '1020', '105', '439', '914', '1099', '1065', '373', '166', '396', '837', '685', '226', '956', '70', '604', '1067', '118', '774', '521', '910', '975', '352', '761', '1007', '428', '1038', '50', '838', '126', '1078', '944', '532', '1041', '540', '128', '722', '478', '639', '668', '343', '1021', '552', '848', '74', '520', '1032', '615', '536', '1117', '646', '390', '923', '194', '216', '99', '372', '857', '335', '505', '972', '727', '1064', '740', '182', '316', '706', '193', '882', '932', '174', '440', '965', '1140', '273', '878', '826', '596', '1123', '1031', '942', '239', '1005', '139', '11', '855', '836', '97', '124', '568', '387', '171', '644', '695', '12', '739', '732', '1035', '242', '441', '928', '71', '248', '794', '458', '915', '475', '1050', '546', '556', '619', '721', '614', '626', '404', '759', '334', '486', '153', '5', '1010', '272', '219', '125', '222', '1098', '758', '213', '235', '916', '253', '573', '636', '1053', '927', '398', '231', '502', '410', '4', '895', '34

In [40]:
dataset = ImageDataset(image_names=train_image_names, 
                       root_dir=root_dir, 
                       labels=label_model_predictions, 
                       target_dists=probs_dev, 
                       transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0)

dev_dataset = ImageDataset(image_names=dev_image_names_modified, 
                           root_dir=validation_root_dir, 
                           labels=Y_dev, 
                           target_dists=probs_dev,
                           transform=transform)
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=False, num_workers=0)

In [ ]:
train_iter = iter(dataloader)

images, label, target_dists, tile_number, image_paths = next(train_iter)

In [ ]:
tile_number

A basic ResNet

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch
import torch.nn.functional as F

def expected_cross_entropy_loss(logits, target_distributions):
    """
    Computes the expected cross-entropy loss for a batch of predictions and target distributions.

    Parameters:
    logits (torch.Tensor): The raw output from the model of shape (batch_size, num_classes).
    target_distributions (torch.Tensor): The target class distributions of shape (batch_size, num_classes),
                                         where each row is a probability distribution over classes.

    Returns:
    torch.Tensor: The expected cross-entropy loss.
    """
    # Convert logits to log probabilities
    log_probs = F.log_softmax(logits, dim=1)
    
    # Compute the element-wise product between target distributions and log probabilities
    # Then, sum across classes to get the cross-entropy for each instance
    cross_entropy = -torch.sum(target_distributions * log_probs, dim=1)
    
    # Take the mean over the batch
    loss = cross_entropy.mean()
    
    return loss


In [42]:
from tqdm import tqdm
def calculate_metrics(y_true, y_pred, abstain_class=-1):
    # Filter out samples where prediction is -1
    # valid_indices = y_pred != abstain_class
    # y_true_filtered = y_true[valid_indices]
    # y_pred_filtered = y_pred[valid_indices]

    # Compute metrics
    conf_matrix = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    return {
        'Confusion Matrix': conf_matrix,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Accuracy': accuracy
    }


# Function to evaluate the model on the dev set
def evaluate(model, dev_loader, device):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient computation
        for images, labels, target_dist, _, _ in tqdm(dev_loader):
            labels =  labels.to(device)
            list_of_images = [image.to(device) for image in images]

            # Forward pass to get outputs
            outputs = model(list_of_images)

            # Get predictions (class with the highest score)
            # _, predicted = torch.max(outputs.data, 1)
            predicted = torch.round(outputs)

            # Store true labels and predictions
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    return np.array(all_labels), np.array(all_preds)
    
def train(model, train_loader, val_loader, criterion, optimizer, device, epochs):
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels, target_dist, _, _ in tqdm(train_loader):
            labels, target_dist = labels.to(device), target_dist.to(device)
            # print(labels)
            list_of_images = [image.to(device) for image in images]

            optimizer.zero_grad()  # Clear the gradients

            outputs = model(list_of_images)  # Forward pass
            # print(outputs)
            
            # print(outputs)
            # print(outputs.dtype)
            # print(labels)
            # probs, predicted = torch.max(outputs.data, 1)
            # max_prob = torch.max(probs)
            loss = criterion(outputs.squeeze(), labels.float())  # Compute the loss
            # loss = expected_cross_entropy_loss(outputs, target_dist)

            loss.backward()  # Backward pass
            optimizer.step()  # Update the weights

            # Calculate running loss and accuracy
            running_loss += loss.item()
            
            predicted = torch.round(outputs.data)
            # print(predicted)
            
            # print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # break

        epoch_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")
        labels, predictions = evaluate(model, val_loader, device)
        # print(labels)
        # print(predictions)

        # Calculate and print precision, recall, and F1-score
        # calculate_metrics(labels, predictions)
        metrics = calculate_metrics(labels, predictions)
        for metric, value in metrics.items():
            print(f"{metric}: {value}")


In [43]:
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

resnet_extractor = ResNetFeatureExtractor()
mlp_classifier = MLPClassifier(input_size=6*2048, hidden_size=512, output_size=1)

model = Gully_Classifier(input_size=6*2048, hidden_size=512, output_size=1).to(device)
# model = Gully_Classifier(input_size=6*2048, hidden_size=512, output_size=1)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# Loss and optimizer
criterion = nn.BCELoss()  # Binary classification loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Only optimize the MLP parameters

# Train the model
epochs = 10
train(model, dataloader, dev_dataloader, criterion, optimizer, device, epochs)


/home/macula/SMATousi/.conda/envs/snorkel/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/macula/SMATousi/.conda/envs/snorkel/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|                                                                                                                   | 0/1065 [00:00<?, ?it/s]/tmp/ipykernel_3257828/1289081151.py:72: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the curr

Epoch [1/10], Loss: 0.5384, Accuracy: 406.72%


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:33<00:00,  1.16it/s]


Confusion Matrix: [[ 94  40]
 [ 52 125]]
Precision: 0.7575757575757576
Recall: 0.7062146892655368
F1 Score: 0.7309941520467836
Accuracy: 0.7041800643086816


  0%|                                                                                                                   | 0/1065 [00:00<?, ?it/s]/tmp/ipykernel_3257828/1289081151.py:72: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images_1 = [imageio.imread(img_path).astype('uint8') for img_path in self.data[idx]]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1065/1065 [17:06<00:00,  1.04it/s]


...

Confusion Matrix: [[ 65  69]
 [ 22 155]]
Precision: 0.6919642857142857
Recall: 0.8757062146892656
F1 Score: 0.773067331670823
Accuracy: 0.707395498392283


  0%|                                                                                                                   | 0/1065 [00:00<?, ?it/s]/tmp/ipykernel_3257828/1289081151.py:72: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images_1 = [imageio.imread(img_path).astype('uint8') for img_path in self.data[idx]]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1065/1065 [17:28<00:00,  1.02it/s]


Epoch [10/10], Loss: 0.3204, Accuracy: 439.70%


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:27<00:00,  1.41it/s]

Confusion Matrix: [[ 82  52]
 [ 40 137]]
Precision: 0.7248677248677249
Recall: 0.7740112994350282
F1 Score: 0.7486338797814208
Accuracy: 0.7041800643086816


Evaluation ...

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Function to evaluate the model on the dev set
def evaluate(model, dev_loader, device):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient computation
        for images, labels, target_dist in tqdm(dev_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass to get outputs
            outputs = model(images)

            # Get predictions (class with the highest score)
            _, predicted = torch.max(outputs.data, 1)

            # Store true labels and predictions
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    return np.array(all_labels), np.array(all_preds)

# Function to calculate precision, recall, and F1 score
def calculate_metrics(labels, predictions):
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return precision, recall, f1

In [ ]:
dev_dataset = ImageDataset(image_names=dev_image_names, root_dir=root_dir, labels=Y_dev, target_dists=probs_dev, transform=transform)
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=False, num_workers=4)

In [ ]:
labels, predictions = evaluate(model, dev_dataloader, device)

# Calculate and print precision, recall, and F1-score
# calculate_metrics(labels, predictions)
metrics = calculate_metrics(labels, predictions)
for metric, value in metrics.items():
    print(f"{metric}: {value}")

Training the CLIP+MLP model